In [1]:
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
from slsim.Pipelines.skypy_pipeline import SkyPyPipeline
from astropy.units import Quantity
from slsim.lens_pop import LensPop
from slsim.lens import Lens
from slsim.Plots.lens_plots import LensingPlots
import numpy as np
from slsim.selection import deflector_cut
import corner
from lenstronomy.Util import constants
from colossus.cosmology import cosmology
import numpy.random as random
from slsim.Util import param_util
import sys
sys.path.append('../slsim/Deflectors')  # 別ディレクトリのパスを追加
from deflectors_base import DeflectorsBase
from lenstronomy.Util import constants
from velocity_dispersion import vel_disp_composite_model

# Galaxy-galaxy simulations

This notebook walks through the basics of simulating a galaxy-galaxy strong lensing population.
The underlying 

assumptions of the galaxy populations (for both lenses and sources) are drawn from a population pre-configured

 and rendered through [SkyPy](https://github.com/skypyproject/skypy). The specific settings are described in the [readme file](https://github.com/LSST-strong-lensing/slsim/tree/gg-lensing/data/SkyPy).

The notebook goes in three steps:

1. The populations of lenses and sources is produced.
2. Random draws of the population are generated and realized as images
3. The full population is generated in catalogue form
4. the full population is represented in a corner plot



## Generate population of galaxies and (potential) deflectors
The LensPop() class in the slsim package is used to produce a set of galaxies (as lenses and sources)

as seen on the sky within a certain sky area.
We use the default SkyPy configuration file. Alternative configuration

files can be used.

In [4]:
# define a cosmology

cosmo_col = cosmology.setCosmology('planck18')
cosmo = FlatLambdaCDM(H0=cosmo_col.H0, Om0=cosmo_col.Om0)

# define a sky area
sky_area = Quantity(value=0.1, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

skypy_config='/Users/ktabe/Research/strong-lens-SNe/slsim/data/SkyPy/lsst-like.yml'
# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_type="all-galaxies",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config=skypy_config,
    sky_area=sky_area,
    cosmo=cosmo,
)

In [5]:
filters=None
pipeline = SkyPyPipeline(
                skypy_config=skypy_config,
                sky_area=sky_area,
                filters=filters,
                cosmo=cosmo,
            )

In [7]:
gal_list = pipeline.red_galaxies
gal_list.sort("z")
gal_list.reverse()

## Elliptical galaxies class

In [5]:
from slsim.Deflectors.elliptical_lens_galaxies import (
                EllipticalLensGalaxies)
kwargs_mass2light = {}
lens_galaxies = EllipticalLensGalaxies(
                pipeline.red_galaxies,
                kwargs_cut=kwargs_deflector_cut,
                kwargs_mass2light=kwargs_mass2light,
                cosmo=cosmo,
                sky_area=sky_area,
            )

In [2]:
import numpy.random as random
from slsim.Deflectors.elliptical_lens_galaxies import (
                elliptical_projected_eccentricity)
index = random.randint(0, lens_galaxies._num_select - 1)
deflec = lens_galaxies._galaxy_select[index]
a,b,c,d = elliptical_projected_eccentricity(**deflec)
print(deflec),print(a,b,c,d)

NameError: name 'lens_galaxies' is not defined

## Compound class

In [41]:
class CompoundLensHalosGalaxies(DeflectorsBase):
    """Class describing compound lens model based on the halo model."""

    def __init__(self, halo_galaxy_list, kwargs_cut, kwargs_mass2light, cosmo, sky_area):
        """

        :param halo_galaxy_list: list of dictionary with lens parameters of
            elliptical dark matte haloes and galaxies (currently supporting SL-Hammocks pipelines)
        :param kwargs_cut: cuts in parameters: band, band_mag, z_min, z_max
        :type kwargs_cut: dict
        # :param kwargs_mass2light: mass-to-light relation
        :param cosmo: astropy.cosmology instance
        :type sky_area: `~astropy.units.Quantity`
        :param sky_area: Sky area over which galaxies are sampled. Must be in units of
            solid angle.
        ## MEMO: DeflectorsBase's inputs are deflector_table, kwargs_cut, cosmo, sky_area
        """
        super().__init__(
            deflector_table=halo_galaxy_list,
            kwargs_cut=kwargs_cut,
            cosmo=cosmo,
            sky_area=sky_area,
        )

        n = len(halo_galaxy_list)
        column_names = halo_galaxy_list.columns
        if "vel_disp" not in column_names:
            halo_galaxy_list["vel_disp"] = -np.ones(n)
        if "mag_g" not in column_names:
            halo_galaxy_list["mag_g"] = -np.ones(n)
        if "mag_r" not in column_names:
            halo_galaxy_list["mag_r"] = -np.ones(n)
        if "mag_i" not in column_names:
            halo_galaxy_list["mag_i"] = -np.ones(n)
        if "mag_z" not in column_names:
            halo_galaxy_list["mag_z"] = -np.ones(n)
        if "mag_Y" not in column_names:
            halo_galaxy_list["mag_Y"] = -np.ones(n)
        if "e1_light" not in column_names or "e2_light" not in column_names:
            halo_galaxy_list["e1_light"] = -np.ones(n)
            halo_galaxy_list["e2_light"] = -np.ones(n)
        if "e1_mass" not in column_names or "e2_mass" not in column_names:
            halo_galaxy_list["e1_mass"] = -np.ones(n)
            halo_galaxy_list["e2_mass"] = -np.ones(n)
        # TODO: check

        # num_total = len(halo_galaxy_list)
        # z_min, z_max = 0, np.max(halo_galaxy_list["z"])

        self._galaxy_select = deflector_cut(halo_galaxy_list, **kwargs_cut)
        # Currently only supporting redshift cut
        self._num_select = len(self._galaxy_select)
        self._cosmo = cosmo

        # TODO: random reshuffle of matched list

    def draw_deflector(self):
        """
        :return: dictionary of complete parameterization of deflector
        """
        cosmo = self._cosmo
        index = random.randint(0, self._num_select - 1)
        deflector_ori = self._galaxy_select.iloc[index]
        deflector = deflector_ori.copy()
        # if deflector["vel_disp"] == -1:
        theta_eff = deflector['tb']/0.551 # [arcsec]
        reff = (theta_eff*cosmo.angular_diameter_distance(deflector['z'])*constants.arcsec).value # physical Mpc
        hubble = cosmo.H0.value/100.
        vel_disp = vel_disp_composite_model(
                theta_eff, deflector['m_gal']/(hubble), reff, max(deflector['m_halo'],deflector['m_acc'])/(hubble),deflector['con'], cosmo, deflector['z'])
        print(vel_disp)
            # vel_disp = vel_disp_composite_model(theta_eff, deflector['mlens_gal']/(hubble), reff, deflector['mlens_halo']/(hubble),deflector['lens_con'], deflector['z'], deflector['zs'], cosmo)
            # deflector["vel_disp"] = vel_disp
        if deflector["mag_g"] == -1 or deflector["mag_r"] or  deflector["mag_i"] == -1 or deflector["mag_z"] or deflector["mag_Y"] == -1:
            mag_g, mag_r, mag_i, mag_z, mag_Y = 0,0,0,0,0 # TODO: make function if needed
        if deflector["e1_light"] == -1 or deflector["e2_light"] == -1:
            e1_light, e2_light, e1_mass, e2_mass = elliptical_projected_eccentricity_galaxy(
                **deflector
            ) # TODO: make function if needed
            deflector["e1_light"] = e1_light
            deflector["e2_light"] = e2_light
            deflector["e1_mass"] = e1_mass
            deflector["e2_mass"] = e2_mass
        return deflector

    def deflector_number(self):
        """

        :return: number of deflectors
        """
        number = self._num_select
        return number


def elliptical_projected_eccentricity_galaxy(elip_lens_gal, **kwargs): #TODO: add row of elip_lens_gal if needed
    """Projected eccentricity of elliptical galaxies as a function of other deflector
    parameters.

    :param ellipticity: eccentricity amplitude
    :type ellipticity: float [0,1)
    :param kwargs: deflector properties
    :type kwargs: dict
    :return: e1_light, e2_light,e1_mass, e2_mass eccentricity components
    """
    ellipticity = elip_lens_gal
    e_light = param_util.epsilon2e(ellipticity)
    phi_light = np.random.uniform(0, np.pi)
    e1_light = e_light * np.cos(phi_light)
    e2_light = e_light * np.sin(phi_light)
    e_mass = 0.5 * ellipticity + np.random.normal(loc=0, scale=0.1)
    phi_mass = phi_light + np.random.normal(loc=0, scale=0.1)
    e1_mass = e_mass * np.cos(phi_mass)
    e2_mass = e_mass * np.sin(phi_mass)
    return e1_light, e2_light, e1_mass, e2_mass


def vel_disp_composite_model2(r, m_star, rs_star, m_halo, c_halo, cosmo, z_lens):
    """Computes the luminosity weighted velocity dispersion for a deflector with a
    stellar Hernquist profile and a NFW halo profile, assuming isotropic anisotropy.

    :param r: radius of the luminosity-weighted velocity dispersion [arcsec]
    :param m_star: stellar mass [M_sun]
    :param rs_star: stellar half light radius [physical Mpc]
    :param m_halo: Halo mass [physical M_sun]
    :param c_halo: halo concentration
    :param cosmo: cosmology
    :type cosmo: ~astropy.cosmology class
    :param z_lens: redshift of the deflector
    :return: velocity dispersion [km/s]
    """
    kwargs_model = {
        "mass_profile_list": ["HERNQUIST", "NFW"],
        "light_profile_list": ["HERNQUIST"],
        "anisotropy_model": "const",
    }

    # turn physical masses to lenstronomy units
    from lenstronomy.Cosmo.lens_cosmo import LensCosmo

    lens_cosmo = LensCosmo(z_lens=z_lens, z_source=z_lens+0.5, cosmo=cosmo)
    # Hernquist profile
    sigma0, rs_angle_hernquist = lens_cosmo.hernquist_phys2angular(
        mass=m_star, rs=rs_star
    )

    rs_angle_nfw, alpha_Rs = lens_cosmo.nfw_physical2angle(M=m_halo, c=c_halo)
    kwargs_mass = [
        {"sigma0": sigma0, "Rs": rs_angle_hernquist, "center_x": 0, "center_y": 0},
        {"alpha_Rs": alpha_Rs, "Rs": rs_angle_nfw, "center_x": 0, "center_y": 0},
    ]
    kwargs_light = [{"amp": 1, "Rs": rs_angle_hernquist, "center_x": 0, "center_y": 0}]
    kwargs_anisotropy = {"beta": 0}

    from lenstronomy.GalKin.numeric_kinematics import NumericKinematics

    kwargs_numerics = {
        "interpol_grid_num": 1000,
        "log_integration": True,
        "max_integrate": 100,
        "min_integrate": 0.0001,
        "max_light_draw": None,
        "lum_weight_int_method": True,
    }
    kwargs_cosmo = {"d_d": lens_cosmo.dd, "d_s": lens_cosmo.ds, "d_ds": lens_cosmo.dds}


    num_kin = NumericKinematics(kwargs_model, kwargs_cosmo, **kwargs_numerics)
    vel_disp = num_kin.lum_weighted_vel_disp(
        r, kwargs_mass, kwargs_light, kwargs_anisotropy
    )
    return vel_disp



In [4]:
import pandas as pd
# columns = [
#     'nim', 'mori', 'mobs', 'sep', 'ein', 'fs', 'frac_sh_trunc', 'mmin', 'flag_out', 'zs', 'xs', 'ys'
# ]

# for i in range(5):
#     suffix = "_sh" if i == 0 else "_sat" if i == 1 else "_hh" if i == 2 else "_cen" if i == 3 else "_pert"
#     columns.extend([
#         f'm_type{suffix}', f'zl{suffix}', f'mass{suffix}', f'xl{suffix}',
#         f'yl{suffix}', f'elip{suffix}', f'pol{suffix}', f'param1{suffix}'
#     ])
# file_path = '/Users/ktabe/Research/strong-lens-SNe/genmock_halo/result/default_kapst/'
# prefix = 'default_LSST_kapst'
# data = np.loadtxt(file_path+"/"+prefix+"_result.dat")
# df = pd.DataFrame(data, columns=columns)
# lens_class = df
# lens_class
df = pd.read_csv("/Users/ktabe/Research/strong-lens-SNe/slsim_KTA/data/SL-Hammocks/qso_salpeter_LSST_multi.csv")

In [17]:
import os
import slsim
path = os.getcwd()
module_path, _ = os.path.split(path)
slhammocks_config = os.path.join(path, "../data/SL-Hammocks/qso_salpeter_LSST_multi.csv")
df = pd.read_csv(slhammocks_config)

In [18]:
mlens_halo = np.where(df["mass_sh"]==-1, df["mass_hh"], df["mass_sh"])
mlens_gal = np.where(df["mass_sh"]==-1, df["mass_cen"], df["mass_sat"])
lens_con = np.where(df["mass_sh"]==-1, df["param1_hh"], df["param1_sh"])
lens_tb = np.where(df["mass_sh"]==-1, df["param1_cen"], df["param1_sat"])
elip_lens_gal = np.where(df["mass_sh"]==-1, df["elip_cen"], df["elip_sat"])
df["mlens_halo"]=mlens_halo
df["mlens_gal"]=mlens_gal
df["lens_tb"]=lens_tb
df["lens_con"]=lens_con
df["elip_lens_gal"] = elip_lens_gal
df = df.rename(columns={'zl_hh': 'z'})

In [2]:
sys.path.append('../slsim/Pipelines')  # 別ディレクトリのパスを追加
from sl_hammocks_pipeline import SLHammocksPipeline

In [11]:
# define a cosmology

cosmo_col = cosmology.setCosmology('planck18')
cosmo = FlatLambdaCDM(H0=cosmo_col.H0, Om0=cosmo_col.Om0)
sky_area = Quantity(value=0.1, unit="deg2")
kwargs_deflector_cut = {"z_min": 0.01, "z_max": 2.5}

In [6]:
pipeline2 = SLHammocksPipeline(
                slhammocks_config=None,
                sky_area=sky_area,
                cosmo=cosmo,
            )

In [20]:
n = len(halo_galaxy_list)
column_names = halo_galaxy_list.columns
if "vel_disp" not in column_names:
    halo_galaxy_list["vel_disp"] = -np.ones(n)
else:
    print("include")

include


In [23]:
kwargs_deflector_cut = {"z_min": 0.01, "z_max": 2.5}
galaxy_select = deflector_cut(halo_galaxy_list, **kwargs_deflector_cut)
galaxy_select

NameError: name 'halo_galaxy_list' is not defined

### Test vel_disp_compound

In [45]:
pipeline2._pipeline.columns

Index(['Unnamed: 0', 'index', 'z', 'm_halo', 'm_acc', 'e_h', 'p_h', 'con',
       'm_gal', 'e_g', 'p_g', 'tb', 'vel_disp'],
      dtype='object')

In [24]:
import numpy.random as random
import sys

kwargs_mass2light = {}
lens_galaxies = CompoundLensHalosGalaxies(
                pipeline2._pipeline.copy(),
                kwargs_cut=kwargs_deflector_cut,
                kwargs_mass2light=kwargs_mass2light,
                cosmo=cosmo,
                sky_area=sky_area,
            )

In [25]:
lens_galaxies._galaxy_select

,Unnamed: 0,index,z,m_halo,m_acc,e_h,p_h,con,m_gal,e_g,...,vel_disp,mag_g,mag_r,mag_i,mag_z,mag_Y,e1_light,e2_light,e1_mass,e2_mass
0,0,1.0,0.019,1.244515e+12,0.000000e+00,0.187930,-144.374000,8.977181,4.517617e+10,0.298916,...,128.801900,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
100,100,101.0,0.144,1.339274e+12,0.000000e+00,0.247963,112.869200,11.008350,4.194567e+10,0.025451,...,115.253096,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
200,200,201.0,0.167,3.723917e+11,0.000000e+00,0.271244,-9.635914,8.594441,8.539675e+09,0.429663,...,70.951111,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
300,300,301.0,0.184,2.403459e+11,7.791242e+11,0.322615,-111.397300,31.934680,4.313094e+10,0.431950,...,173.523664,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
400,400,401.0,0.213,4.813280e+10,1.539985e+11,0.293230,-98.551830,22.060570,3.007909e+09,0.021218,...,70.217042,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836100,836100,836101.0,1.990,1.320623e+12,0.000000e+00,0.361820,-125.420200,1.948017,3.128009e+10,0.319185,...,136.727897,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
836200,836200,836201.0,1.993,4.675144e+10,1.679422e+11,0.151623,113.741200,12.795430,1.465869e+09,0.424344,...,92.630333,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
836300,836300,836301.0,1.995,4.027170e+11,0.000000e+00,0.239865,21.876960,3.597686,3.384027e+09,0.265425,...,77.866145,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
836400,836400,836401.0,1.998,4.989141e+10,1.814349e+11,0.584665,-31.060090,9.095567,1.346018e+09,0.349045,...,60.557533,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [52]:
# test = lens_galaxies.draw_deflector()
i=120
deflector = lens_galaxies._galaxy_select.iloc[i]

theta_eff = deflector['tb']/0.551 # [arcsec]
reff = (theta_eff*cosmo.angular_diameter_distance(deflector['z'])*constants.arcsec).value # physical Mpc
hubble = cosmo.H0.value/100.

vel_disp = vel_disp_composite_model(theta_eff, deflector['m_gal']/(hubble), reff, max(deflector['m_halo'],deflector['m_acc'])/(hubble), deflector['con'], cosmo, deflector['z'])
vel_disp2 = vel_disp_composite_model2(theta_eff, deflector['m_gal']/(hubble), reff, deflector['m_halo']/(hubble), deflector['con'], cosmo, deflector['z'])

# re_pop_sub = (deflector['tb']*theta_eff*cosmo.angular_diameter_distance(deflector['z'])*constants.arcsec).value

# vel_disp_composite_model(deflector['tb']/0.551, deflector['m_g'].iloc[i]/(cosmo.H0/100.),
#                                 re_pop_sub, max(df_sub['m_h'].iloc[i],df_sub['m_acc'].iloc[i])/(cosmo.H0/100.),
#                                 df_sub['con'].iloc[i], df_sub['zl'].iloc[i], max(1.5,df_sub['zl'].iloc[i]+0.5),cosmop18))
#     vel_pop_sub_ar = np.array(vel_pop_sub)
print(deflector['m_halo'],deflector['m_acc'],vel_disp,vel_disp2, lens_galaxies._galaxy_select.iloc[i]["vel_disp"],lens_galaxies._galaxy_select.iloc[i],cosmo.angular_diameter_distance(deflector['z']))
# # # test

796159400000.0 0.0 128.4151557557737 128.4125275148259 128.39161113322533 Unnamed: 0    1.200000e+04
index         1.200100e+04
z             7.420000e-01
m_halo        7.961594e+11
m_acc         0.000000e+00
e_h           3.716677e-01
p_h           1.407213e+02
con           1.135522e+01
m_gal         2.634698e+10
e_g           3.702175e-01
p_g           8.901383e+01
tb            8.547288e-02
vel_disp      1.283916e+02
mag_g        -1.000000e+00
mag_r        -1.000000e+00
mag_i        -1.000000e+00
mag_z        -1.000000e+00
mag_Y        -1.000000e+00
e1_light     -1.000000e+00
e2_light     -1.000000e+00
e1_mass      -1.000000e+00
e2_mass      -1.000000e+00
Name: 12000, dtype: float64 1551.5006031719888 Mpc


In [46]:
hubble

0.6766

0.0897516624218156

In [147]:
index = random.randint(0, lens_galaxies._num_select - 1)
deflec = lens_galaxies._galaxy_select[index]
a,b,c,d = elliptical_projected_eccentricity(**deflec)
print(deflec),print(a,b,c,d)

lens_class.(by='', ascending=True, inplace=True)

In [ ]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# define a sky area
sky_area = Quantity(value=0.1, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_type="halomodel",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config='/Users/ktabe/Research/strong-lens-SNe/slsim/data/SkyPy/lsst-like.yml',
    sky_area=sky_area,
    cosmo=cosmo,
)

In [4]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# define a sky area
sky_area = Quantity(value=0.1, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens = Lens(
    source_dict,
    deflector_dict,
    cosmo,
    deflector_type="EPL",
    source_type="extended",
    lens_equation_solver="lenstronomy_analytical",
    variability_model=None,
    kwargs_variability=None,
    sn_type=None,
    sn_absolute_mag_band=None,
    sn_absolute_zpsys=None,
    test_area=4 * np.pi,
    mixgauss_means=None,
    mixgauss_stds=None,
    mixgauss_weights=None,
    magnification_limit=0.01,
    light_profile="single_sersic",
    lightcurve_time=None,
    deflector_type="all-galaxies",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config='/Users/ktabe/Research/strong-lens-SNe/slsim/data/SkyPy/lsst-like.yml',
    sky_area=sky_area,
    cosmo=cosmo,
)

TypeError: Lens.__init__() got an unexpected keyword argument 'deflector_type'

## Generate images of random lenses
The LensingPlots() class has the functionality to draw random lenses and makes an image of it.
Currently

default settings in lenstronomy are chosen for the LSST image settings. These will be able to be replaced with the

LSST simulation tools.

In [ ]:
# make some cuts in the image separations and limited magnitudes of the arc
kwargs_lens_cut_plot = {
    "min_image_separation": 0.8,
    "max_image_separation": 10,
    "mag_arc_limit": {"g": 23, "r": 23, "i": 23},
}


gg_plot = LensingPlots(gg_lens_pop, num_pix=64, coadd_years=10)

# generate montage indicating which bands are used for the rgb color image
fig, axes = gg_plot.plot_montage(
    rgb_band_list=["i", "r", "g"],
    add_noise=True,
    n_horizont=5,
    n_vertical=2,
    kwargs_lens_cut=kwargs_lens_cut_plot,
)
plt.show()

## Generate the full population
We are using the instance of the LensPop() class to draw the full population within specified cuts in a Monte Carlo process.

In [ ]:
# specifying cuts of the population
kwargs_lens_cuts = {"mag_arc_limit": {"g": 28}}
# drawing population
gg_lens_population = gg_lens_pop.draw_population(kwargs_lens_cuts=kwargs_lens_cuts)

## Represent key quantities of full population in corner plots
We calculate few key quantities of the lenses. The full population is represented each with a Lens() class

object that allows to compute and return these (and more) quantities.

In [ ]:
print("Number of lenses:", len(gg_lens_population))

lens_samples = []
labels = [
    r"$\sigma_v$",
    r"$\log(M_{*})$",
    r"$\theta_E$",
    r"$z_{\rm l}$",
    r"$z_{\rm s}$",
    r"$m_{\rm source}$",
    r"$m_{\rm lens}$",
]

for gg_lens in gg_lens_population:
    vel_disp = gg_lens.deflector_velocity_dispersion()
    m_star = gg_lens.deflector_stellar_mass()
    theta_e = gg_lens.einstein_radius
    zl = gg_lens.deflector_redshift
    zs = gg_lens.source_redshift
    source_mag = gg_lens.extended_source_magnitude(band="g", lensed=True)
    deflector_mag = gg_lens.deflector_magnitude(band="g")
    lens_samples.append(
        [vel_disp, np.log10(m_star), theta_e, zl, zs, source_mag, deflector_mag]
    )

In [ ]:
hist2dkwargs = {
    "plot_density": False,
    "plot_contours": False,
    "plot_datapoints": True,
    "color": "b",
    "data_kwargs": {"ms": 5},
}
corner.corner(np.array(lens_samples), labels=labels, **hist2dkwargs)
plt.show()